In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from pathlib import Path

In [3]:
# TODO: PATH 설정

# main project path
PROJECT_ROOT = Path('/content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation')
os.chdir(PROJECT_ROOT) # 현재 경로 수정
# print(f"현재 작업 경로: {os.getcwd()}")

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"
DOMAIN_JSON = DATA_ROOT / "domain_knowledge.json"
MMAD_JSON = DATA_ROOT / "mmad.json"
META_CSV = DATA_ROOT / "metadata.csv"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# check (선택사항)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Root: {DATA_ROOT}")
print(f"Config Root: {CONFIG_ROOT}")
print(f"Current working directory: {os.getcwd()}")

Project Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation
Data Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation/dataset/MMAD
Config Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation/configs
Current working directory: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation


In [4]:
!python ./scripts/setup_conda.py

🌟 Anomalib 통합 설치 

1️⃣ Miniconda 설치 중...

🏃 실행 중: wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh

🏃 실행 중: bash /tmp/miniconda.sh -b -p /content/conda
PREFIX=/content/conda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /content/conda

🏃 실행 중: /content/conda/bin/pip install uv -q

2️⃣ Anaconda ToS 동의 및 설정...

🏃 실행 중: 
    source /content/conda/bin/activate
    conda config --set solver classic 2>/dev/null || true
    echo "y" | /content/conda/bin/conda tos accept --override-channels --

In [5]:
import subprocess

# 가상환경 내 파이썬 경로
PY = "/content/conda/envs/anomaly_report/bin/python"

# 확인할 패키지 목록
target_packages = [
    "anomalib", "lightning", "torchmetrics",
    "numpy", "torch", "torchvision", "opencv-python", "pillow", "scikit-learn", "scikit-image",
    "pyyaml", "tqdm", "matplotlib", "seaborn", "pandas"
]

print("========================================================================")
print("🧐 anomaly_report 환경 내부 패키지 점검 결과")
print("========================================================================")

# pip list를 통해 설치된 패키지 확인
try:
    result = subprocess.check_output([PY, "-m", "pip", "list"], text=True)
    installed_list = result.lower()

    for pkg in target_packages:
        # opencv는 패키지명이 opencv-python이므로 체크 방식 조정
        check_name = "opencv-python" if pkg == "opencv-python" else pkg.lower()

        if check_name in installed_list:
            # 실제 설치된 버전 행 추출
            version_line = [line for line in result.split('\n') if line.lower().startswith(check_name)][0]
            print(f"✅ {version_line}")
        else:
            print(f"❌ {pkg} : 설치되지 않음")

except Exception as e:
    print(f"에러 발생: {e}")

print("========================================================================")

🧐 anomaly_report 환경 내부 패키지 점검 결과
✅ anomalib               1.1.0
✅ lightning              2.1.4
✅ torchmetrics           1.2.1
✅ numpy                  1.26.4
✅ torch                  2.1.2+cu118
✅ torchvision            0.16.2+cu118
✅ opencv-python-headless 4.10.0.84
✅ pillow                 12.0.0
✅ scikit-learn           1.3.2
✅ scikit-image           0.21.0
✅ PyYAML                 6.0.2
✅ tqdm                   4.66.5
✅ matplotlib             3.8.4
✅ seaborn                0.13.2
✅ pandas                 2.2.2


In [18]:
!/content/conda/bin/uv pip install open_clip_torch FrEIA einops timm kornia imgaug omegaconf rich --python /content/conda/envs/anomaly_report/bin/python

Using Python 3.10.19 environment at: /content/conda/envs/anomaly_report
Resolved 64 packages in 187ms
Prepared 4 packages in 68ms
Installed 4 packages in 5ms
 + ftfy==6.3.1
 + open-clip-torch==3.2.0
 + regex==2026.1.15
 + wcwidth==0.5.3


In [40]:
%%bash
/content/conda/envs/anomaly_report/bin/python <<EOF

import os
from pathlib import Path

# --- [초강수 1: Lightning 콜백 무력화] ---
import lightning.pytorch.callbacks as callbacks
class FakeProgressBar(callbacks.Callback): pass
callbacks.RichProgressBar = FakeProgressBar

# --- [초강수 2: Anomalib 내부 rich 유틸리티 무력화] ---
import anomalib.utils.rich as anomalib_rich
from unittest.mock import MagicMock

# 에러가 발생한 'CacheRichLiveState'를 아무 일도 안 하는 객체로 교체
anomalib_rich.CacheRichLiveState = MagicMock()
# 진행 상황을 추적하는 'safe_track'이 rich를 안 쓰고 그냥 루프만 돌게 교체
anomalib_rich.safe_track = lambda sequence, *args, **kwargs: sequence
# ----------------------------------------------

from anomalib.data import MVTec
from anomalib.models import Patchcore
from anomalib.engine import Engine

# --- [사용자 설정 구간] ---
DATA_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/dataset/MMAD/MVTec-AD')
CATEGORY = "bottle"
RESULT_DIR = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/notebooks/noh/results')

def main():
    datamodule = MVTec(
        root=DATA_ROOT,
        category=CATEGORY,
        train_batch_size=32,
        eval_batch_size=32,
        image_size=(256, 256),
        num_workers=2
    )

    model = Patchcore(
        backbone="wide_resnet50_2",
        layers=["layer2", "layer3"]
    )

    engine = Engine(
        task="segmentation",
        default_root_dir=RESULT_DIR / CATEGORY,
        enable_progress_bar=False,
        logger=CSVLogger(save_dir=RESULT_DIR / CATEGORY, name="logs")
    )

    print(f"🚀 [{CATEGORY}] 내부 유틸리티까지 모두 패치했습니다. 분석 시작...")

    try:
        engine.fit(model=model, datamodule=datamodule)
        results = engine.test(model=model, datamodule=datamodule)

        print("\n" + "="*50)
        print(f"✅ {CATEGORY} 최종 분석 결과:")
        print(results)
        print("="*50)

    except Exception as e:
        print(f"❌ 실행 중 에러 발생: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

EOF

🚀 [bottle] 내부 유틸리티까지 모두 패치했습니다. 분석 시작...
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                 │ PatchcoreModel           │ 24.9 M │
│ 1 │ _transform            │ Compose                  │      0 │
│ 2 │ normalization_metrics │ MinMax                   │      0 │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │
└───┴───────────────────────┴──────────────────────────┴────────┘
Trainable params: 24.9 M                                                        
Non-trainable params: 0                                                         
Total params: 24.9 M                                                   

/content/conda/envs/anomaly_report/lib/python3.10/site-packages/lightning/fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.
Could not find wandb. To use this feature, ensure that you have wandb installed.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility